In [1]:
from googleads import adwords
import pandas as pd

In [2]:
def get_campaignId(campaign):
    campaign_service = client.GetService('CampaignService', version='v201802')
    query = (adwords.ServiceQueryBuilder()
           .Select('Id')
           .Where('CampaignName').EqualTo(campaign)
           .OrderBy('Name')
           .Limit(0, 10)
           .Build())
    for page in query.Pager(campaign_service):
        if 'entries' in page:
            for camp in page['entries']:
                print(camp['id'])
        else:
            print('no campaign found')

In [15]:
def get_adgroupId(adgroup, client):
    adgroup_service = client.GetService('AdGroupService', version='v201802')
    query = (adwords.ServiceQueryBuilder()
           .Select('Id')
           .Where('AdGroupName').EqualTo(adgroup)
           .OrderBy('Name')
           .Limit(0, 10)
           .Build())
    for page in query.Pager(adgroup_service):
        if 'entries' in page:
            for adg in page['entries']:
                return(adg['id'])
        else:
            return(0)

In [4]:
# def get_accounts(client):
    
#     managed_customer_service = client.GetService('ManagedCustomerService', version='v201802')
#     selector = {
#                 'fields': ['Name','CustomerId'],
#                 'predicates': [
#                     {
#                         'field': 'Name',
#                         'operator': 'IN',
#                         'values': list(df['AccountDescriptiveName'].unique())
#                     }
#                 ]
#     }
#     accounts = managed_customer_service.get(selector)
#     return accounts

# def upload_keywords(client):
#     accounts = get_accounts(client)

#     try:
#         for entry in accounts['entries']:
#             #iterates through accounts
#             account_name = entry['name']
#             client.client_customer_id = entry['customerId']
#             ad_group_criterion_service = client.GetService('AdGroupCriterionService', version='v201802')
            
#             adgroups = list(df[df['AccountDescriptiveName'] == account_name]['AdGroupId'].unique())
#             for adgroup in adgroups:
                
#                 terms = list(df[(df['AccountDescriptiveName'] == account_name) & (df['AdGroupId'] == adgroup)]['Query'])
#                 for term in terms:
#                     try:
#                         keyword = {
#                             'xsi_type': 'BiddableAdGroupCriterion',
#                             'adGroupId': adgroup,
#                             'criterion': {
#                                 'xsi_type': 'Keyword',
#                                 'matchType': 'EXACT',
#                                 'text': term
#                             },
#                             'finalUrls': {
#                                 'urls': ['https://transferwise.com/?utm_source=google&utm_medium=cpc&matchtype={matchtype}&device={device}&creative={creative}&keyword={keyword}&campaignid={campaignid}&adgroupid={adgroupid}&utm_campaign={campaignid}___{adgroupid}']
#                             }
#                         }

#                         operations = [
#                             {
#                                 'operator': 'ADD',
#                                 'operand': keyword
#                             }
#                         ]
#                         ad_group_criteria = ad_group_criterion_service.mutate(operations)['value']
#                     except Exception,e:
#                         print('Error with ' + term)
#                         print(e)
            
#     except Exception,e:
#         print(e)

In [5]:
def get_exact_names():
    df = pd.read_excel('exact_upload.xlsx')
    df['campaign_exact'] = df['CampaignName'].apply(lambda x: x.replace('Broad', 'Exact'))
    df['adgroup_exact'] = df['AdGroupName'].apply(lambda x: x.replace('Broad', 'Exact'))
    df.to_excel('exact_upload_clean.xlsx')

In [12]:
def get_ids(client):
    managed_customer_service = client.GetService('ManagedCustomerService', version='v201802')
    selector = {
                'fields': ['Name','CustomerId'],
                'predicates': [
                    {
                        'field': 'Name',
                        'operator': 'IN',
                        'values': list(df['AccountDescriptiveName'].unique())
                    }
                ]
    }
    accounts = managed_customer_service.get(selector)
    account_col = []
    adgroup_id_col = []
    keyword_col = []
    
    try:
        for entry in accounts['entries']:
            account_name = entry['name']
            account_col.append(account_name)
            
            client.client_customer_id = entry['customerId']
            adgroups = list(df[df['AccountDescriptiveName'] == account_name]['adgroup_exact'].unique())
            for adgroup in adgroups:
                adgroup_id_col.append(get_adgroupId(adgroup, client))
                
    except:
        print('error')

In [13]:
if __name__ == '__main__':
    get_exact_names()
    df = pd.read_excel('exact_upload_clean.xlsx')
    df.reset_index(inplace=True)
    adwords_client = adwords.AdWordsClient.LoadFromStorage('../googleads.yaml')
    get_ids(adwords_client)
#     upload_keywords(adwords_client)

Your default encoding, cp1252, is not UTF-8. Please run this script with UTF-8 encoding to avoid errors.


AU | Consumer | SEM | English
51381643589
AU | Consumer | SEM | English
45696080078
All | Consumer | SEM | English
47551345589
